# Economic Freedom and Happiness Across Europe

In [132]:
import pandas as pd
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pycountry 

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Function for converting to iso 
def getIsoCode(country_name):
    try:
        country_iso = pycountry.countries.search_fuzzy(country_name)[0]
        return country_iso.alpha_3
    except LookupError:
        return ""

In [133]:
happiness_data = pd.read_csv("happiness.csv")
happiness_data["Country"] = happiness_data["Country or region"]
happiness_data = happiness_data.drop(columns=["Country or region", "Overall rank"])
happiness_data = happiness_data.rename(columns={"Score": "Happiness Score"})

In [134]:
freedom_data = pd.read_csv("freedom.csv", low_memory=False)
# Normalize country column
freedom_data = freedom_data.rename(columns={"countries": "Country"})

# Filter out years other than 2019
freedom_data["year"] = pd.to_numeric(freedom_data["year"], errors='coerce')
freedom_data = freedom_data[freedom_data["year"] == 2019]

# Reset and Drop
freedom_data = freedom_data.reset_index()
freedom_data = freedom_data.drop(columns="index")

freedom_filtered = freedom_data[freedom_data['Country'].isin([
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", 
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", 
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", 
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", 
    "Liechtenstein", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", 
    "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", 
    "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Vatican City"
])]

columns_to_keep = [
    'Country', 'ef_government', 'ef_gender', 'ef_legal', 'ef_money', 'ef_trade', 'ef_regulation', 'ef_rank'
]

freedom_filtered = freedom_filtered[columns_to_keep]

In [135]:
data = pd.merge(happiness_data, freedom_filtered, on='Country')
data = data.drop_duplicates(subset=['Country'])

data["iso"] = data['Country'].apply(getIsoCode)

# CREATING NEW RANK
sorted_df = data.sort_values(by='ef_rank')
sorted_df['ef_rank'] = range(1, len(sorted_df) + 1)

# Display the sorted DataFrame with ranks
data = sorted_df

In [139]:
color_scale = "Viridis"  # Example color scale

fig = px.choropleth(
    data,
    locations="iso",
    title='Economic Freedom by Rank',
    color="ef_rank",
    hover_name="iso",
    hover_data=['iso'],
    color_continuous_scale=color_scale,
    projection="mercator",  # Ensure to use Mercator projection for global maps
)

fig.update_geos(
    projection_type="natural earth",  # Set the projection type
    center=dict(lon=15, lat=54),  # Set the center of the map to Europe
    scope="europe",  # Set the scope to Europe
    showcoastlines=True,  # Show coastlines for better geographic context
)

fig.update_geos(
    fitbounds="locations",  # Adjust the map bounds to fit the specified locations
)
# Set center to Europe
fig.update_geos(center=dict(lon=15, lat=54))

fig.show()